In [14]:
import slopestabilitytools
import slopestabilityML
import pandas as pd
import settings
settings.init()

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers
from keras.utils import to_categorical
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [12]:
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature

def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("CLASSN")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

In [13]:
test_results = slopestabilitytools.datamanagement.import_tests()
is_success = slopestabilitytools.folder_structure.create_folder_structure()

test_training_combined = pd.DataFrame()
test_prediction_combined = pd.DataFrame()

test_training, test_prediction = slopestabilityML.split_dataset(test_results.keys(), 999)

for name in test_training:
    test_training_combined = test_training_combined.append(test_results[name])
test_training_combined = test_training_combined.reset_index()
test_training_combined = test_training_combined.drop(['index'], axis='columns')
#x_train, y_train = slopestabilityML.preprocess_data(test_results_combined)
test_training_combined = test_training_combined.drop(['NAME', 'Z', 'INM', 'RES', 'INMN', 'CLASS', 'LABELS' ], axis=1)

for name in test_prediction:
    test_prediction_combined = test_prediction_combined.append(test_results[name])
test_prediction_combined = test_prediction_combined.reset_index()
test_prediction_combined = test_prediction_combined.drop(['index'], axis='columns')
#x_train, y_train = slopestabilityML.preprocess_data(test_results_combined)
test_prediction_combined = test_prediction_combined.drop(['NAME', 'Z', 'INM', 'RES', 'INMN', 'CLASS', 'LABELS' ], axis=1)


train_ds = dataframe_to_dataset(test_training_combined)
val_ds = dataframe_to_dataset(test_prediction_combined)

train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

Folder for figures exists!
Folder for figures exists!
Folder for figures exists!
Folder for figures exists!
Folder for figures exists!
Number of tests used for prediction: 3
Number of tests used for training: 7


In [15]:
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature

In [23]:
X = keras.Input(shape=(1,), name="X", dtype="float")
Y = keras.Input(shape=(1,), name="Y", dtype="float")
RES = keras.Input(shape=(1,), name="RESN", dtype="float")
SEN = keras.Input(shape=(1,), name="SEN", dtype="float")
#CLASS = keras.Input(shape=(1,), name="CLASSN", dtype="float")

all_inputs = [X, Y, RES, SEN]

X_encoded = encode_numerical_feature(X, "X", train_ds)
Y_encoded = encode_numerical_feature(Y, "Y", train_ds)
RES_encoded = encode_numerical_feature(RES, "RESN", train_ds)
SEN_encoded = encode_numerical_feature(SEN, "SEN", train_ds)

all_features = layers.concatenate([X_encoded, Y_encoded, RES_encoded, SEN_encoded])


20/04/21 - 16:09:33 - tensorflow - WARNING - AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7fe87c4783b0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
20/04/21 - 16:09:34 - tensorflow - WARNING - AutoGraph could not transform <function encode_numerical_feature.<locals>.<lambda> at 0x7fe87c4ba290> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
20/04/21 - 16:09:35 - tensorflow - WARNING - AutoGraph could

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `

In [24]:
x = layers.Dense(32, activation="relu")(all_features)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(all_inputs, output)
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [25]:
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
1816/1816 [==============================] - 12s 6ms/step - loss: -102.0956 - accuracy: 0.1333 - val_loss: -963.8683 - val_accuracy: 0.1944

In [ ]:
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=8))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [9]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
model.fit(x_train, y_train, epochs=20)


ValueError: Error when checking input: expected dense_1_input to have shape (8,) but got array with shape (3,)